## Импорт библиотек и загрузка данных

In [ ]:
# Установка библиотек

!pip install streamlit -q


In [ ]:
!pip install streamlit-folium -q

In [ ]:
# Импорт библиотек
from math import sqrt
from streamlit_folium import st_folium

import joblib
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
import streamlit as st

from joblib import dump
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Скачиваем необходимые данные с Google Drive

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=12UNNxWzKJ0RmQxaQp3acMHvXl7pKWv-N' -O main_data.csv -q

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=153FC-BlQVJoV_uMr1RPxaxt710TyD-qi' -O app.py -q

In [ ]:
!mkdir pages

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=15-fvjGDq7nXrvkt6LMwTlU1QJ3jb0FrG' -O cpages/Покупка.py -q

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1-BnRXpEd2l7yLaSnbOGOZSir-l1nmn9f' -O content/pages/Продажа.py -q

In [ ]:
# Загрузка данных

df = pd.read_csv('main_data.csv')

df.head()

## Анализ данных

In [ ]:
# Размер датафрейма

df.shape

In [ ]:
df.info()

In [ ]:
# Явные дубликаты

df.duplicated().sum()

In [ ]:
# Количество пропущенных значений

sorted(df.isna().sum().items(), key=lambda x: x[1])

In [ ]:
# Изучим целевую переменную

df.boxplot('price');

In [ ]:
# Обрежем выбросы - оставим только те значения, что попадают в квантиль 0.90

df = df[df.price < df.price.quantile(q=0.90)]
df.shape

In [ ]:
msno.matrix(df)
plt.show()

In [ ]:
# Вычисление корреляционной матрицы

correlation_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, cmap='viridis', vmin=-1, vmax=1, center=0, annot=True, fmt=".2f")
plt.show()

In [ ]:
# Создание столбца "construction_year" из "build_date" или "complitation_year" и удаление старых столбцов

df['construction_year'] = df['build_date'].combine_first(df['complitation_year'])
df = df.drop(['build_date', 'complitation_year'], axis=1)

In [ ]:
# Заполнение пропущенных значений в столбце 'isСomplete' на основе значений в столбце 'construction_year'

df['isСomplete'] = df.apply(lambda row: 0 if pd.isnull(row['isСomplete']) and row['construction_year'] > 2024 else
                                    1 if pd.isnull(row['isСomplete']) and row['construction_year'] < 2024 else
                                    row['isСomplete'], axis=1)

In [ ]:
print("Nan: ", df['isСomplete'].isnull().sum())
print("Nan: ", df['construction_year'].isnull().sum())

### Жилая площадь и количество комнат

In [ ]:
df = df.drop(['kitchen_area', 'living_area'], axis=1)

In [ ]:
# Рассчитываем среднюю площадь для каждого количества комнат

average_areas = {}
for i in range(1, 11):
    total_area_sum = df[df['rooms_count'] == i]['total_area'].sum()
    count = df[df['rooms_count'] == i]['total_area'].count()
    if count > 0:
        average_areas[i] = total_area_sum / count

print("Средняя площадь для квартир с разным количеством комнат:")
for key, value in average_areas.items():
    print(f"{key} комнатная квартира: {value:.2f} кв.м")

In [ ]:
# Заполняем пропуски в количестве комнат исходя из средней площади

for index, row in df.iterrows():
    if pd.isnull(row['rooms_count']):
        min_diff = float('inf')
        for key, value in average_areas.items():
            if abs(row['total_area'] - value) < min_diff:
                min_diff = abs(row['total_area'] - value)
                df.at[index, 'rooms_count'] = key

In [ ]:
df = df.drop(['parking', 'decoration', 'balcony'], axis=1)

In [ ]:
df = df.drop(['passenger_elevator', 'cargo_elevator', 'address', 'metro', 'metro_distance', 'metro_transport', 'house_material',  'is_apartments'], axis=1)

In [ ]:
df = df.dropna(subset=['district'])

In [ ]:
df = df.dropna(subset=['isСomplete'])

In [ ]:
msno.matrix(df)
plt.show()

In [ ]:
df.hist(figsize=(15,12))

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(x='longitude', y='latitude', data = df, hue='price', palette='coolwarm')

In [ ]:
df['region'].unique()

In [ ]:
df = pd.get_dummies(df, columns=['region'])

In [ ]:
df.head()

In [ ]:
# Вычисление корреляционной матрицы

correlation_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, cmap='viridis', vmin=-1, vmax=1, center=0, annot=True, fmt=".2f")
plt.show()

In [ ]:
df = df.drop(['district'], axis=1)

## Обучение модели

In [ ]:
# Разделение данных на признаки и целевую переменную
X = df.drop('price', axis=1)
y = df['price']

In [ ]:
# Разделение на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Инициализация и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Предсказание на тестовой выборке
predictions = model.predict(X_test)

In [ ]:
# Оценка качества модели
r2 = r2_score(y_test, predictions)
print("Коэффициент детерминации (R^2):", r2)

In [ ]:
# Обучение модели Random Forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

In [ ]:
# Предсказание на тестовых данных
y_pred_rf = rf.predict(X_test)

In [ ]:
# Оценка качества модели Random Forest
r2_rf = r2_score(y_test, y_pred_rf)
print("Коэффициент детерминации Random Forest (R^2):", r2_rf)

In [ ]:
import xgboost as xgb

# Инициализация и обучение модели градиентного бустинга
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_xgb = xgb_model.predict(X_test)

# Оценка качества модели градиентного бустинга
r2_xgb = r2_score(y_test, predictions_xgb)
print("Коэффициент детерминации градиентного бустинга (R^2):", r2_xgb)

In [ ]:
# Улучшение гиперпараметров модели градиентного бустинга
xgb_model_tuned = xgb.XGBRegressor(n_estimators=1000, max_depth=5, learning_rate=0.1, colsample_bytree=0.5)
xgb_model_tuned.fit(X_train, y_train)

# Предсказание на тестовой выборке с улучшенными гиперпараметрами
predictions_xgb_tuned = xgb_model_tuned.predict(X_test)

# Оценка качества модели градиентного бустинга с улучшенными гиперпараметрами
r2_xgb_tuned = r2_score(y_test, predictions_xgb_tuned)
print("Коэффициент детерминации градиентного бустинга с улучшенными гиперпараметрами (R^2):", r2_xgb_tuned)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Инициализация и обучение модели KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_knn = knn.predict(X_test)

# Оценка качества модели KNeighborsRegressor
r2_knn = r2_score(y_test, predictions_knn)
print("Коэффициент детерминации KNeighborsRegressor (R^2):", r2_knn)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Инициализация и обучение модели деревьев принятия решений
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_dt = dt.predict(X_test)

# Оценка качества модели деревьев принятия решений
r2_dt = r2_score(y_test, y_pred_dt)
print("Коэффициент детерминации деревьев принятия решений (R^2):", r2_dt)

In [ ]:
import matplotlib.pyplot as plt

# Названия моделей
models = ['Градиентный бустинг\nс улучшенными параметрами', 'Градиентный бустинг', 'Random Forest', 'Дерево принятия решений', 'KNeighborsRegressor', 'Линейная регрессия']

# Коэффициенты детерминации
r2_scores = [r2_xgb_tuned, r2_xgb, r2_rf, r2_dt, r2_knn, r2]

colors = ['skyblue', 'lightcoral', 'lightgreen', 'lightsalmon', 'lightblue', 'pink']

plt.figure(figsize=(10, 6))
plt.bar(models, r2_scores, color=colors)
plt.xlabel('Модели')
plt.ylabel('Коэффициент детерминации (R^2)')
plt.title('Сравнение качества различных моделей')
plt.ylim(0.7, 0.95) # установите нужный диапазон для y-оси
plt.xticks(rotation=45)
plt.show()

In [ ]:
dump(rf, 'model_rf.joblib')

## Веб-интерфейс

In [ ]:
# Запускает приложение streamlit (файл приложения app.py)

!streamlit run app.py & npx localtunnel --port 8501

In [ ]:
# Запускаем утилиту wget для получения внешнего IP-адреса от сервиса ipv4.icanhazip.com

!wget -q -O - ipv4.icanhazip.com